In [ ]:
%matplotlib inline
import numpy as np
import math
import csv
import re
import nltk
from PIL import Image
import os
from os import path
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
    nltk.download("words")

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def split(word): 
    return list(word) 

def process_document(text,emoji):
    # Convert words to lower case

    text = text.lower()
    
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()) 
    text = text.split()

    newtext = [x for x in text if not x.startswith('@') and not x.startswith('https')] ##remove @ and link
    stop_words = stopwords.words('english')
    newtext = [x for x in text if x not in stop_words] ## remove stopwords
    
    newtext = ' '.join(newtext)
    
    tokenizer = RegexpTokenizer(r'\w+')
    newtext = tokenizer.tokenize(newtext)
    
    stemmer = SnowballStemmer('english')
    newtext = list(map(stemmer.stem, newtext))
    emojis = split(emoji)
    newtext = newtext + emojis
    label = 0
    for ele in newtext:
        if ele =='amp':
            label = 1

    newtext = ' '.join(newtext)
    if label == 1:
        newtext = newtext.replace('amp','and')
   
    
    return newtext


def run(link):
    with open(link,encoding="utf8") as words_file:
        
        csv_reader = csv.DictReader(words_file, delimiter = ',')
        data = []
        emoji = []
        for row in csv_reader:
            data.append(row['Text'])
            emoji.append(row['Emoji'])
        data = np.array(data)
    words =[]
    for i, tweet in enumerate(data):
        #tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
        a = process_document(tweet, emoji[i])
        words += a
    counts = {}
    for i in words:
        counts[i] = counts.get(i, 0) + 1
    sorted_dict = sorted(counts.items(), key=lambda x: x[1])
    top_words = np.array(sorted_dict[-50:][::-1])[:,0]
    
    
    top_words = ' '.join(top_words)
    #print(emoji)
    
    d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
    
    trump_mask = np.array(Image.open(path.join(d, "dt.png")))
    wc = WordCloud(background_color="white", max_words=2000, mask=trump_mask, contour_width=3, contour_color='steelblue').generate(top_words)
    wc.to_file(path.join(d, "Trump.jpeg"))

    
    
    plt.figure(figsize=(30,25))
    plt.axis("off")
    plt.figure()
    plt.imshow(ds_mask, cmap=plt.cm.gray, interpolation='bilinear')
    plt.axis("off")
    plt.show()

    
def main():
    
    run('trump.csv')
    #Hint from: https://blog.emojipedia.org/emoji-sentiment-analysis-2015-2017/
    

    

    
if __name__ == '__main__':
	main()